In [244]:
import numpy as np
rand = np.random.default_rng(42)

In [245]:
class Layer:
    def __init__(self):
        pass 

    def __call__(self,x,train=False):
        pass 

    def backward(self,out_grad,learning_rate):
        pass 

In [246]:
class Linear(Layer):
    def __init__(self,fan_in,fan_out,seed=42):
        self.fan_in = fan_in 
        self.fan_out = fan_out

        limit = np.sqrt(6 / (fan_in + fan_out))
        self.weights = np.random.uniform(-limit, limit, size=(fan_out, fan_in))
        # self.weights = np.random.randn(fan_out,fan_in)
        self.bias = np.ones((fan_out,1))
        # print(self.weights.shape)
        # print(self.grad.shape)
    def __call__(self,x,train=False):
        self.input = x.reshape(self.fan_in,-1)
        return np.dot(self.weights,self.input) + self.bias
    

    def __repr__(self):
        pass 

    def backward(self, out_grad, learning_rate ):
        wgrad = np.dot(out_grad, self.input.T) 
        bgrad = out_grad 
        inputgrad = np.dot(self.weights.T, out_grad)

        self.weights -= learning_rate * wgrad 
        self.bias -= learning_rate * bgrad 

        # print(f"{self.wgrad}")
        # print(f"{self.bgrad}")

        return inputgrad 

    def reset_grad(self):
        self.wgrad = np.zeros((self.fan_in,self.fan_out))
        self.bgrad = np.zeros((1,self.fan_out)) 


In [247]:
class Softmax(Layer):
    def __call__(self, input, train=False):
        self.input = input
        tmp = input - max(input)  
        tmp = np.exp(tmp)
        self.output = tmp / np.sum(tmp)
        return self.output 
    def backward(self, out_grad, learning_rate):
        n = np.size(self.output) 
        return np.dot((np.identity(n)-self.output.T) * self.output, out_grad)  

In [248]:
class Activation(Layer):
    def __init__(self,activation,activation_prime):
        self.activation = activation 
        self.activation_grad = activation_prime

    def __call__(self, input, train=False):
        self.input = input 
        return self.activation(self.input) 
    
    def backward(self, out_grad, learning_rate):
        return np.multiply(out_grad, self.activation_grad(self.input))

In [249]:
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)
        def tanh_grad(x):
            return 1-np.tanh(x)**2 
        super().__init__(tanh,tanh_grad)

In [250]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1.0 / (1.0 + np.exp(-x))
        
        def sigmoid_grad(x):
            return sigmoid(x) * (1.0 - sigmoid(x)) 
        super().__init__(sigmoid,sigmoid_grad) 
            

In [251]:
class ReLU(Activation):
    def __init__(self):
        def relu(x):
            return np.maximum(0,x)
        def relu_grad(x):
            return np.where(x > 0, 1, np.where(x < 0, 0, 0.5))
        super().__init__(relu,relu_grad)

In [252]:
class Dropout(Layer):
    def __init__(self, dropout_rate):
        self.dropout_rate = dropout_rate
        self.mask = None

    def __call__(self, x, train=False):
        if train:
            self.mask = (np.random.rand(*x.shape) < (1 - self.dropout_rate)) / (1 - self.dropout_rate)
            # print(self.mask)
            return x * self.mask
        else:
            return x

    def backward(self, grad, learning_rate):
        return grad * self.mask if self.mask is not None else grad

In [253]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_grad(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_grad(y_true, y_pred): # wrt y_pred
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

def cross_entropy(y_true,y_pred):
    return np.mean(-y_true*np.log(y_pred)) 

def cross_entropy_grad(y_true,y_pred):
    return np.mean(-y_true/y_pred) 

In [254]:
class NN:
    def __init__(self, *layers):
        self.layers : Layer = [] 
        for layer in layers:
            self.layers += [layer]
        
    def __call__(self,input,train=False):
        for layer in self.layers:
            input = layer(input,train=train)
        return input 
    def backward(self, ):
        pass 
    
    def save(self,filename):
        pass 
    def load(self,filename):
        pass
    
    def train_minibatch(self,):
        pass 
    
    def train_batch(self, loss, loss_grad, X, y, epochs = 1000, learning_rate = 0.001, verbose=True):
        for epoch in range(epochs):
            error = 0 
            for x, y_ in zip(X,y):
                output = self.__call__(x,train=True)
                
                y__ = y_.reshape(-1,1)
                error += loss(y__,output)

                grad = loss_grad(y__,output)

                for layer in reversed(self.layers):
                    grad = layer.backward(grad, learning_rate)
            error /= len(X)
            if verbose:
                print(f"{epoch=}, {error=}")

    def eval(self, loss, X,y):
        error = 0 
        for x,y in zip(X,y):
            output = self.__call__(x)
            error += loss(y,output)
        error /= len(X)
        return error 
    


In [255]:
lin = Linear(10,5)
relu = ReLU() 
sigmoid = Sigmoid()
softmax = Softmax()

model = NN(lin, sigmoid, relu, softmax) 

x = rand.random(10).reshape(-1,1)

print(x)
model(x)

[[0.77395605]
 [0.43887844]
 [0.85859792]
 [0.69736803]
 [0.09417735]
 [0.97562235]
 [0.7611397 ]
 [0.78606431]
 [0.12811363]
 [0.45038594]]


array([[0.20106312],
       [0.2037229 ],
       [0.19253629],
       [0.18725386],
       [0.21542383]])

In [256]:
model = NN(Linear(32,5), ReLU(), Linear(5,10), Softmax()) 

x = np.random.randn(1,32)

print(x)
result = model(x) 
print(result, np.sum(result))


[[ 1.32619805  1.89564367 -0.46529648 -0.24996919  0.35606244 -0.28221255
  -0.63704127 -1.04219965 -0.31773012 -0.82844622  0.81787209  0.55289724
   0.1380805   1.84531836 -1.14168622 -1.53733126  0.41455222 -1.24364498
   0.1708396   0.77796014 -1.22170123 -0.2083445  -0.68965957  3.19704106
  -0.23796433  0.08749685  0.33976824  0.4568267  -1.85720938  0.35361752
   0.88832478  1.29159404]]
[[0.00677718]
 [0.01277286]
 [0.22919295]
 [0.01829279]
 [0.03795207]
 [0.00547625]
 [0.01020633]
 [0.60284563]
 [0.05204314]
 [0.0244408 ]] 1.0


Import Dataset

In [257]:
from torchvision import datasets, transforms 

train_validation_dataset = datasets.EMNIST(root='./data', 
                                           split='letters',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True,
                                        )


independent_test_dataset = datasets.EMNIST(
                             root='./data',
                             split='letters',
                             train=False,
                             transform=transforms.ToTensor(),
                             )


In [259]:
type(train_validation_dataset)
n_classes = len(train_validation_dataset.classes)
n_datapoints = len(train_validation_dataset.targets)

X_train = np.array(train_validation_dataset.data.reshape(-1,28*28))
# y_train = np.zeros((len(train_validation_dataset.targets),len(train_validation_dataset.classes)))

y_train = np.array([[1 if temp == i else 0 for i in range(n_classes)] for temp in train_validation_dataset.targets])
y_train[3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [261]:
X_test = np.array(independent_test_dataset.data.reshape(-1,28*28))
y_test = np.array([[1 if temp == i else 0 for i in range(n_classes)] for temp in independent_test_dataset.targets])

In [263]:
import pandas as pd 

df = pd.DataFrame(train_validation_dataset.data.reshape(-1,28*28))

In [264]:
df[:5]

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [265]:
model = NN(Linear(28, 26), Dropout(0.5))

In [268]:
model = NN(Linear(28*28,1024), ReLU(), Dropout(0.2), Linear(1024,n_classes), Softmax(), Dropout(0.3), Sigmoid())

In [269]:
model.train_batch(cross_entropy, cross_entropy_grad, X_train, y_train, 3)

epoch=0, error=0.007693150207259264
epoch=1, error=0.007740308105596984
epoch=2, error=0.0077259086732750825


In [273]:
model(X_test[8])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [271]:
y_test[8]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])